In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime as dt
from scipy.spatial import distance
import scipy.stats as stats

%matplotlib inline

In [2]:
TIME_SAMPLE_FREQ = '30s'

In [3]:
df = pd.read_csv(".\\_generated\\wifi_1.data", index_col = False, header = None, low_memory = False, \
                 names = ['timestamp', 'uuid', 'bssid', 'ssid', 'capabilities', \
                          'freq0', 'freq1', 'chwidth', 'freq', 'level', 'operator', \
                          'time', 'venue', '802', 'passpoint'])

In [4]:
df.head()

,timestamp,uuid,bssid,ssid,capabilities,freq0,freq1,chwidth,freq,level,operator,time,venue,802,passpoint
0,06.12.2020_18:08:47.262,00bf7e9c-735f-44f4-a819-e09975ca9bcb,34:ce:00:4d:64:1d,Hello_ World!,[WPA-PSK-TKIP+CCMP][WPA2-PSK-TKIP+CCMP][RSN-PS...,0,0.0,0.0,2422.0,-26.0,NaN,1.094820e+12,NaN,False,False
1,06.12.2020_18:08:47.262,00bf7e9c-735f-44f4-a819-e09975ca9bcb,34:ce:00:4d:64:1e,Hello_ World!_5G,[WPA-PSK-TKIP+CCMP][WPA2-PSK-TKIP+CCMP][RSN-PS...,5775,0.0,2.0,5805.0,-32.0,NaN,1.094823e+12,NaN,False,False
2,06.12.2020_18:08:47.262,00bf7e9c-735f-44f4-a819-e09975ca9bcb,40:b4:f0:28:29:c1,Moscow_WiFi_Free,[ESS][WFA-HT],5230,0.0,1.0,5220.0,-37.0,NaN,1.094820e+12,NaN,False,False
3,06.12.2020_18:08:47.262,00bf7e9c-735f-44f4-a819-e09975ca9bcb,40:b4:f0:28:29:c3,Beeline_Free,[ESS][WFA-HT],5230,0.0,1.0,5220.0,-37.0,NaN,1.094820e+12,NaN,False,False
4,06.12.2020_18:08:47.262,00bf7e9c-735f-44f4-a819-e09975ca9bcb,40:b4:f0:28:29:c0,Moscow_WiFi_Free,[ESS][WFA-HT],0,0.0,0.0,2462.0,-37.0,NaN,1.094822e+12,NaN,False,False


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 595134 entries, 0 to 595133
Data columns (total 15 columns):
timestamp       595134 non-null object
uuid            595134 non-null object
bssid           595134 non-null object
ssid            586151 non-null object
capabilities    595130 non-null object
freq0           595130 non-null object
freq1           595130 non-null float64
chwidth         595130 non-null float64
freq            595130 non-null float64
level           595130 non-null float64
operator        580843 non-null float64
time            595130 non-null float64
venue           1 non-null float64
802             595129 non-null object
passpoint       595130 non-null object
dtypes: float64(7), object(8)
memory usage: 68.1+ MB


In [6]:
df['timestamp'] = df['timestamp'].apply(lambda x: dt.strptime(x, '%d.%m.%Y_%H:%M:%S.%f'))

In [7]:
df.index = pd.DatetimeIndex(df.timestamp)
df = df.sort_index()

In [8]:
df = df.drop(['timestamp', 'chwidth', 'operator', 'venue', '802', 'time', 'ssid', 'capabilities', \
              'passpoint', 'freq0', 'freq1'], axis = 1)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 595134 entries, 2020-12-06 17:56:05.536000 to 2020-12-10 21:50:56.753000
Data columns (total 4 columns):
uuid     595134 non-null object
bssid    595134 non-null object
freq     595130 non-null float64
level    595130 non-null float64
dtypes: float64(2), object(2)
memory usage: 22.7+ MB


In [10]:
bssid_map = { bssid.replace(' ', ''): idx for bssid, idx in zip(df.bssid.unique(), range(len(df.bssid.unique()))) }

In [11]:
df.bssid = df.bssid.apply(lambda x: str(x).replace(' ', ''))
df.level = df.level.apply(lambda x: str(x).replace(' ', ''))
df.freq = df.freq.apply(lambda x: str(x).replace(' ', ''))

In [12]:
df['bssid_level'] = df[['bssid', 'level']].agg(','.join, axis=1)
df['count'] = 1

In [13]:
def agg_string_join(col):
    col = col.apply(lambda x: str(x))
    return col.str.cat(sep = ',').replace(' ', '')

In [14]:
def agg_bssid_col(col):
    array_len = len(bssid_map)
    array = np.zeros(array_len, dtype = 'float')
    def fill_array(x):
        tmp = x.split(',')
        bssid = tmp[0]
        level = float(tmp[1])
        array[bssid_map[bssid.replace(' ', '')]] = level
        return
        
    col.apply(lambda x: fill_array(x))
    return np.array2string(array, separator = ',').replace(' ', '')[1:-1]

In [15]:
all_func_dicts_quantum = { 'freq': agg_string_join, 'level': agg_string_join, 'bssid_level' : agg_bssid_col, 'count' : 'sum' }

In [16]:
df_quantum = df.groupby(['timestamp', 'uuid'], as_index=True).agg(all_func_dicts_quantum)

In [17]:
df_quantum = df_quantum.reset_index()
df_quantum.index = pd.DatetimeIndex(df_quantum.timestamp)

In [18]:
df_quantum = df_quantum[df_quantum['count'] != 0]

In [19]:
df_conn = pd.read_csv(".\\_generated\\conn_wifi_5.data", index_col = False, header = None, low_memory = False, \
                 names = ['timestamp', 'uuid', 'stamp', 'bssid', '1', '2', '3', '4', '5', '6', 'level', '8'])

df_conn = df_conn.drop(df_conn.columns.difference(['bssid','timestamp', 'level']), axis = 1)
df_conn['timestamp'] = df_conn['timestamp'].apply(lambda x: dt.strptime(x, '%d.%m.%Y_%H:%M:%S.%f'))
df_conn.index = pd.DatetimeIndex(df_conn.timestamp)
df_conn = df_conn.sort_index()

In [20]:
def get_level_from_row(row):
    bssid = df_conn.iloc[df_conn.index.get_loc(row.name, method = 'nearest')]['bssid']
    if str(bssid) == 'nan' or str(bssid) == 'null' or str(bssid) == '':
        return 0
    
    level = df_conn.iloc[df_conn.index.get_loc(row.name, method = 'nearest')]['level']
    time = df_conn.iloc[df_conn.index.get_loc(row.name, method = 'nearest')]['timestamp']
    return level if abs((time - row.name).total_seconds()) <= 10 else 0


df_quantum['conn_level'] = df_quantum.apply(lambda row: get_level_from_row(row), axis = 1)

In [21]:
def string2array(string):
    try:
        array = np.fromstring(string, sep=',')
        return array
    except:
        return np.nan

def to_ones_array(array):
    try:
        array[array != 0] = 1
        return array
    except:
        return np.nan

def get_len(obj):
    try:
        length = len(obj)
        return length
    except:
        return np.nan

In [22]:
def get_occured_nets_count(row, prev_col, curr_col):
    prev = to_ones_array(string2array(row[prev_col]))
    curr = to_ones_array(string2array(row[curr_col]))
    intersection = np.logical_and(curr, prev)
    diff = np.logical_and(curr, np.logical_not(intersection))
    
    if (np.count_nonzero(np.logical_or(prev, curr)) == 0):
        return 0
    
    return np.count_nonzero(diff) / np.count_nonzero(np.logical_or(prev, curr))

def get_disappeared_nets_count(row, prev_col, curr_col):
    prev = to_ones_array(string2array(row[prev_col]))
    curr = to_ones_array(string2array(row[curr_col]))
    intersection = np.logical_and(curr, prev)
    diff = np.logical_and(prev, np.logical_not(intersection))
    
    if (np.count_nonzero(np.logical_or(prev, curr)) == 0):
        return 0
    
    return np.count_nonzero(diff) / np.count_nonzero(np.logical_or(prev, curr))

def get_jaccard_index(row, prev_col, curr_col):
    prev = to_ones_array(string2array(row[prev_col]))
    curr = to_ones_array(string2array(row[curr_col]))
    return distance.jaccard(prev, curr)

def get_occur_speed(row, prev_col, curr_col):
    prev = to_ones_array(string2array(row[prev_col]))
    curr = to_ones_array(string2array(row[curr_col]))
    return np.linalg.norm(prev - curr) / np.sqrt(get_len(prev))
    
def get_level_speed(row, prev_col, curr_col):
    prev = string2array(row[prev_col])
    curr = string2array(row[curr_col])
    return np.linalg.norm(prev - curr) / np.sqrt(get_len(prev))

def calc_single_cols_in_window(df, col, new_col, window, func):
    def func_wrapper(func, row, prev_col, curr_col):
        delta = row.timestamp - row.prev_timestamp
        if pd.isnull(delta):
            delta = 0
        else:
            delta = abs(delta.total_seconds())
        if delta > 10 * 60:
            return np.nan
        else:
            return func(row, prev_col_name, col)
        
    new_cols = []
        
    for i in range(window):
        prev_col_name = "_".join(['prev', col, str(i + 1)])
        new_col_name = "_".join([new_col, str(i + 1)])
        
        df['prev_timestamp'] = df.timestamp.shift(i + 1)
        df[prev_col_name] = df[col].shift(i + 1)
        df[new_col_name] = df.apply(lambda row: func_wrapper(func, row, prev_col_name, col), axis = 1)
        df = df.drop(prev_col_name, axis = 1)
        df = df.drop('prev_timestamp', axis = 1)
        new_cols.append(new_col_name)
        
    df["_".join([new_col, 'mean'])] = df[new_cols].mean(axis = 1)
    df["_".join([new_col, 'median'])] = df[new_cols].median(axis = 1)
    df["_".join([new_col, 'var'])] = df[new_cols].var(axis = 1)
        
    return df

In [23]:
WINDOW_SIZE = 3

occur_and_level_columns_map = [
    ("bssid_level", "occured_nets_count", WINDOW_SIZE, get_occured_nets_count),
    ("bssid_level", "disappeared_nets_count", WINDOW_SIZE, get_disappeared_nets_count),
    ("bssid_level", "jaccard_index", WINDOW_SIZE, get_jaccard_index), 
    ("bssid_level", "occur_speed", WINDOW_SIZE, get_occur_speed),
    ("bssid_level", "level_speed", WINDOW_SIZE, get_level_speed)
]

for (col, new_col, window, func) in occur_and_level_columns_map:
    df_quantum = calc_single_cols_in_window(df_quantum, col, new_col, window, func)

In [24]:
def get_conn_level_speed(row, prev_col, curr_col):
    return row[curr_col] - row[prev_col]

In [25]:
WINDOW_SIZE = 3

single_columns_map = [
    ("conn_level", "conn_level_speed", WINDOW_SIZE, get_conn_level_speed),
    ("count", "count_speed", WINDOW_SIZE, get_conn_level_speed)
]

for (col, new_col, window, func) in single_columns_map:
    df_quantum = calc_single_cols_in_window(df_quantum, col, new_col, window, func)

In [26]:
def get_acceleration(row, prev_col, curr_col):
    return abs(row[curr_col] - row[prev_col])

In [27]:
WINDOW_SIZE = 3

multi_speed_cols = ["occured_nets_count_mean", "jaccard_index_mean", "occur_speed_mean", "disappeared_nets_count_mean", 
                    "conn_level_speed_mean", "count_speed_mean"]

for col in multi_speed_cols:
    df_quantum = calc_single_cols_in_window(df_quantum, col, "_".join(["acceleration", col]), window, func)

In [28]:
def agg_str(col):
#     all_freq = col.str.cat(sep=',')
    return string2array(col)

def str_mean(col):
    array = agg_str(col)
    if str(array) == 'nan':
        return 0 
    return np.mean(array)

# def str_var(col):
#     array = agg_str(col)
#     if str(array) == 'nan':
#         return 0 
#     return np.var(array)

# def str_median(col):
#     array = agg_str(col)
#     if str(array) == 'nan':
#         return 0 
#     return np.median(array)

# def str_skew(col):
#     array = agg_str(col)
#     if str(array) == 'nan':
#         return 0 
#     return stats.skew(array)

# def str_kurt(col):
#     array = agg_str(col)
#     if str(array) == 'nan':
#         return 0 
#     return stats.kurtosis(array)

def mean(col):
    return np.mean(col)

def var(col):
    return np.var(col)

def median(col):
    return np.median(col)

def skew(col):
    return stats.skew(col)

def kurt(col):
    return stats.kurtosis(col)

In [29]:
df_quantum['freq'] = df_quantum.apply(lambda row: str_mean(row['freq']), axis = 1)
df_quantum['level'] = df_quantum.apply(lambda row: str_mean(row['level']), axis = 1)

In [30]:
df_quantum = df_quantum.drop(['bssid_level', 'timestamp', 'uuid'], axis = 1)

In [31]:
# string_common_cols = df_quantum.columns[1:3]
common_cols = df_quantum.columns[1:5]
speed_acc_cols = df_quantum.columns[5:]

# string_common_funcs_list = [str_mean, str_var, str_median, str_skew, str_kurt]
common_funcs_list = [mean, var, median, skew, kurt]
special_funcs_list = [mean, pd.DataFrame.mad, skew]

common_cols_map = { col : common_funcs_list for col in common_cols }
# string_common_cols_map = { col : string_common_funcs_list for col in string_common_cols }
speed_acc_cols_map = { col : special_funcs_list for col in speed_acc_cols }

agg_dict = common_cols_map
# agg_dict.update(string_common_cols_map)
agg_dict.update(speed_acc_cols_map)

In [32]:
df_quantum[speed_acc_cols] = df_quantum[speed_acc_cols].apply(pd.to_numeric)

In [33]:
df_sampling = df_quantum.groupby(pd.Grouper(freq = TIME_SAMPLE_FREQ)).agg(agg_dict)

D:\Program Files\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
D:\Program Files\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
D:\Program Files\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3405: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)


In [34]:
df_rolling = df_quantum.rolling(TIME_SAMPLE_FREQ, min_periods = 1, center = False).agg(agg_dict)

In [35]:
df_sampling.columns = ["_".join([str(high_level_name), str(low_level_name)]) \
                       for (high_level_name, low_level_name) in df_sampling.columns.values]

df_rolling.columns = ["_".join([str(high_level_name), str(low_level_name)]) \
                      for (high_level_name, low_level_name) in df_rolling.columns.values]

In [36]:
df_sampling = df_sampling.dropna()
df_sampling = df_sampling.fillna(0)

df_rolling = df_rolling.dropna()
df_rolling = df_rolling.fillna(0)

In [37]:
# df_rolling['day_time'] = df_rolling.apply(lambda x: x.name.hour + x.name.minute / 60 , axis = 1)
# df_sampling['day_time'] = df_sampling.apply(lambda x: x.name.hour + x.name.minute / 60 , axis = 1)

In [38]:
df_sampling.to_csv(".\\_datasets\\5s\\wifi_sampling_dataset_5.csv")
df_rolling.to_csv(".\\_datasets\\5s\\wifi_rolling_dataset_5.csv")